In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report

In [2]:
pos_rev = pd.read_csv('pos.txt', sep='\n', header=None, encoding='latin-1')
pos_rev['mood'] = 1
pos_rev.rename(columns={0:'review'}, inplace=True)
pos_rev.head()

,review,mood
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


In [3]:
neg_rev = pd.read_csv('negative.txt', sep='\n', header=None, encoding='latin-1')
neg_rev['mood'] = 0
neg_rev.rename(columns={0:'review'}, inplace=True)
neg_rev.head()

,review,mood
0,"simplistic , silly and tedious.",0
1,"it's so laddish and juvenile , only teenage bo...",0
2,exploitative and largely devoid of the depth o...,0
3,[garbus] discards the potential for pathologic...,0
4,a visually flashy but narratively opaque and e...,0


In [4]:
data = pd.concat([pos_rev, neg_rev], axis=0).reset_index(drop=True)
data

,review,mood
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1
...,...,...
10657,a terrible movie that some people will neverth...,0
10658,there are many definitions of 'time waster' bu...,0
10659,"as it stands , crocodile hunter has the hurrie...",0
10660,the thing looks like a made-for-home-video qui...,0


In [5]:
sw = stopwords.words('english')

In [6]:
data['review'] = data.review.apply(lambda x : x.lower())
data['review'] = data.review.apply(lambda x : ' '.join([word for word in x.split() if word not in sw]))
data['review'] = data.review.apply(lambda x : ' '.join([word for word in x.split() if word not in string.punctuation]))

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data['review'].values, data['mood'].values, test_size=0.2)

In [8]:
train_data = pd.DataFrame({'review':x_train, 'mood':y_train})
test_data = pd.DataFrame({'review':x_test, 'mood':y_test})

In [9]:
vectoriser = TfidfVectorizer()
train_vectors = vectoriser.fit_transform(train_data['review'])
test_vectors = vectoriser.transform(test_data['review'])

In [10]:
classifier = svm.SVC()
classifier.fit(train_vectors, train_data['mood'])

SVC()

In [11]:
pred = classifier.predict(test_vectors)

In [12]:
report = classification_report(test_data['mood'] , pred , output_dict=True)
print(f"positve {report['1']['recall']}")
print(f"neagtive {report['0']['recall']}")

positve 0.7777777777777778
neagtive 0.75591985428051
